In [ ]:
# Requesting Prompts from CHATGPT using key notions and best performing prompt from original work.
from getpass import getpass
from openai import OpenAI


# Nouns to be used in prompts


csmodl_overlap_keyNotions = [
    "land", 
    "place", 
    "name", 
    "service", 
    "case", 
    "time", 
    "hour"
]


enslaved_overlap_keyNotions = [
    "judge",
    "name",
    "cause",
    "cost",
    "place",
    "fire",
    "interest",
    "library",
    "event",
    "service",
    "researcher",
    "time",
    "material",
    "institution",
    "condition",
    "hour",
    "land",
    "language",
    "case",
    "region",
]


combined_overlap_keyNotions = [
    "land",
    "place",
    "name",
    "service",
    "case",
    "time",
    "hour",
]

dedupe_keyNotions = list(set(csmodl_overlap_keyNotions + enslaved_overlap_keyNotions))

print(f"Total key Notions: {len(csmodl_overlap_keyNotions)+len(enslaved_overlap_keyNotions)}")
print(f"Total number of unique key notions: {len(dedupe_keyNotions)}")

# Top performing prompts from Enslaved Micropatterns (Queries 3, 5, and 7) work
prompts = [
    "Formulate an ontology related to {noun}. Provide it in valid Turtle/RDF format, excluding any extra text.",
    "Construct an ontology focused on the following: {noun}. Provide it in valid Turtle/RDF format, excluding any extra text.",
    "Design an ontology to encompass the following: {noun}.   Provide it in valid Turtle/RDF format, excluding any extra text.",
]

In [4]:
apiKey = getpass("Enter your OpenAI API key:")

In [ ]:

client = OpenAI(api_key=apiKey)
promptStrings = []
responses = []

for notion in csmodl_overlap_keyNotions:
    for prompt in prompts:
        promptString = prompt.format(noun=notion)
        print(f"Prompt: {promptString}")
        promptStrings.append(f"csmodl\t{notion}\t{promptString}")

for notion in enslaved_overlap_keyNotions:
    for prompt in prompts:
        promptString = prompt.format(noun=notion)
        print(f"Prompt: {promptString}")
        promptStrings.append(f"enslaved\t{notion}\t{promptString}")

for notion in combined_overlap_keyNotions:
    for prompt in prompts:
        promptString = prompt.format(noun=notion)
        print(f"Prompt: {promptString}")
        promptStrings.append(f"combined\t{notion}\t{promptString}")


print(f"Total number of prompts: {len(promptStrings)}")



In [ ]:
count = 0
for promptString in promptStrings:
    notionList = promptString.split("\t")[0]
    notion = promptString.split("\t")[1]
    prompt = promptString.split("\t")[2]
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o",
    )
    response = chat_completion.choices[0].message.content.replace("\n", "")
    responses.append(f"{notionList}\t{notion}\t{prompt}\t{response}")
    count += 1
    print(f"Completed {count} of {len(promptStrings)} prompts.")

currResponse = 0
numResponses = len(responses)
with open("responses.tsv", "w") as f:
    for response in responses:
        notionList = response.split("\t")[0]
        notion = response.split("\t")[1]
        prompt = response.split("\t")[2]
        response = response.split("\t")[3]
        f.write(f"{notionList}\t{notion}\t{prompt}\t{response}")
        currResponse += 1
#        print(f"Completed {currResponse} of {numResponses} responses.")
f.close()

In [12]:
with open("responsesV2.tsv", "w") as f:
    for response in responses:
        responseArray = response.split("\t")
        notionList = responseArray[0]
        notion = responseArray[1]
        prompt = responseArray[2]
        response = str(responseArray[3])
        f.write(f"{notionList}\t{notion}\t{prompt}\t{response}\n")
        if prompt.startswith("Formulate an ontology related to"):
            with open(f"./{notionList}_responses/{notion}_formulate.ttl", "w") as g:
                g.write(response)
                g.close()
        elif prompt.startswith("Construct an ontology focused on the following"):
            with open(f"./{notionList}_responses/{notion}_construct.ttl", "w") as g:
                g.write(response)
                g.close()
        elif prompt.startswith("Design an ontology to encompass the following"):
            with open(f"./{notionList}_responses/{notion}_design.ttl", "w") as g:
                g.write(response)
                g.close()
        else:
            print(f"Error: {notionList}: {notion} | {prompt}")